In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import imutils
import numpy as np
#获取视频
cap=cv.VideoCapture('/Users/lixuecheng/Desktop/2021-7-28 15.56拍摄的影片 #2.mov')

In [2]:
#首先读取视频的一帧，确定要选择的区域进行识别
ret,frame=cap.read()
#选择一个区域
r,h,c,w=400,200,600,300
#将窗口的区域赋值给window
window=(c,r,h,w)

In [3]:
#ROI,确定我们的ROI区域
roi=frame[r:r+h,c:c+w]

In [4]:
#将ROI区域转换成HSV格式
hsv_roi=cv.cvtColor(roi,cv.COLOR_BGR2HSV)
#计算直方图
'''
cv2.calcHist(images, channels, mask, histSize, ranges)
 channels 代表色彩通道
 mask一般设置为none
 histSize:使用多少个bin(柱子)，一般为256
 ranges:像素值的范围，一般为[0,255]表示0~255
'''
roi_hist=cv.calcHist(hsv_roi,[0],None,[180],[0,180])

In [5]:
#开始目标追踪
#设置终止条件，最大迭代次数为10，阈值为1
term=(cv.TERM_CRITERIA_EPS|cv.TERM_CRITERIA_COUNT,10,1)

In [6]:
while (True):
    #读取每一帧
    ret,frame=cap.read()
    if ret==True:
        #计算直方图的反射投影
        hsv=cv.cvtColor(frame,cv.COLOR_BGR2HSV)
        '''
        calcBackProject(images, channels, hist, ranges, scale)

        images参数表示输入图像（是HSV图像）。传入时应该用中括号[ ]括起来。
        channels参数表示用于计算反向投影的通道列表，通道数必须与直方图维度相匹配。
        hist参数表示输入的模板图像直方图。
        ranges参数表示直方图中每个维度bin的取值范围 （即每个维度有多少个bin）。
        scale参数表示可选输出反向投影的比例因子，一般取1。
        '''
        dst=cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        ret,trackwindow=cv.CamShift(dst,window,term)
        

        pts=cv.boxPoints(ret)
        pts=np.int0(pts)
        img2=cv.polylines(frame,[pts],True,(0,255,255),2)
        cv.imshow("frame",img2)
        if cv.waitKey(40)&0xFF==27:
            break
cap.release()
cv.destroyAllWindows()